In [1]:
import json
import re
from textblob import TextBlob
import pandas as pd
import numpy as np

Rutas de los datasets

In [2]:
ruta_steam_games = 'datasets/output_steam_games.json'
ruta_users_items = 'datasets/australian_users_items.json'
ruta_users_reviews = 'datasets/australian_user_reviews.json'

ruta_parquet_games = "datasets/data_steam_games.parquet"
ruta_parquet_reviews = "datasets/data_reviews.parquet"
ruta_parquet_items = "datasets/data_items.parquet"

ruta_games_csv = 'datasets/steam_games.csv'
ruta_items_csv = 'datasets/users_items.csv'
ruta_reviews_csv = 'users_reviews.csv'

empezando extracción dataframe steam_games

In [104]:
dic_steam_games = []
with open(ruta_steam_games, 'r') as file:
    for line in file:
        json_data = json.loads(line)
        dic_steam_games.append(json_data)
df_games = pd.DataFrame(dic_steam_games)

In [105]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


Guardando el archivo como CSV

In [107]:
df_games.to_csv(ruta_games_csv, index=False, sep=';', encoding="utf-8")

In [108]:
df_games = pd.read_csv(ruta_games_csv, sep=";")

C:\Users\Leonardo\AppData\Local\Temp\ipykernel_3900\1274100216.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_games = pd.read_csv(ruta_games_csv, sep=";")


Removendo Líneas Duplicadas

In [109]:
df_games = df_games.drop_duplicates()

guardando el archivo como parquet

In [110]:
df_games.to_parquet(ruta_parquet_games)

empezando extracción dataframe user_items

In [111]:
with open(ruta_users_items, 'r', encoding='utf-8') as f:
    data =f.readlines()

records = [eval(line.strip()) for line in data]

df_users_items = pd.DataFrame(records)

# Crear el DataFrame con las columnas user_id, items_count, steam_id y user_url
df_users_info = df_users_items[['user_id', 'items_count', 'steam_id', 'user_url']]

# Crear una lista para almacenar los registros desglosados de items
items_records = []

# Recorrer el DataFrame original y desglosar los elementos de items
for index, row in df_users_items.iterrows():
    user_id = row['user_id']
    items_list = row['items']
    for item in items_list:
        item_record = {
            'user_id': user_id,
            'item_id': item['item_id'],
            'item_name': item['item_name'],
            'playtime_forever': item['playtime_forever'],
            # 'playtime_2weeks': item['playtime_2weeks']
        }
        items_records.append(item_record)

# Crear el DataFrame con los registros desglosados de items
df_items_info = pd.DataFrame(items_records)

In [112]:
df_items_info.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


In [113]:
df_items_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           object
 2   item_name         object
 3   playtime_forever  int64 
dtypes: int64(1), object(3)
memory usage: 157.3+ MB


guardando el archivo como csv

In [114]:
df_items_info.to_csv(ruta_items_csv, index=False, sep=';', encoding='utf-8')

In [115]:
df_items_info = pd.read_csv(ruta_items_csv,sep=";")

Removendo Líneas duplicadas

In [116]:
df_items_info = df_items_info.drop_duplicates()

guardando el archivo como parquet

In [117]:
df_items_info.to_parquet(ruta_parquet_items)

empezando extracción dataframe user_reviews

In [118]:
# Abre el archivo JSON y lee los datos
with open(ruta_users_reviews, 'r', encoding='utf-8') as f:
    data = f.readlines()
# Corrige el formato JSON eliminando comas al final de las líneas y encierra todo en corchetes para formar una lista de diccionarios válida
fixed_data = "[" + ",".join(data) + "]"
# Convierte el JSON en una lista de diccionarios
records = eval(fixed_data)
# Crea el DataFrame con las reviews y agrega la relación con user_id
df_users_reviews = pd.DataFrame([{
    "user_id": record["user_id"],
    "item_id": review["item_id"],
    "posted": review["posted"],
    "recommend": review["recommend"],
    "review": review["review"]
} for record in records for review in record["reviews"]])

In [119]:
df_users_reviews.head()

,user_id,item_id,posted,recommend,review
0,76561197970982479,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,76561197970982479,43110,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,js41637,227300,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...


In [120]:
df_users_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  object
 2   posted     59305 non-null  object
 3   recommend  59305 non-null  bool  
 4   review     59305 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


guardando el archivo como csv

In [121]:
df_users_reviews.to_csv(ruta_reviews_csv, index=False, sep=';', encoding='utf-8')

In [122]:
df_users_reviews = pd.read_csv('users_reviews.csv',sep=";")

Removendo Duplicados

In [123]:
df_users_reviews = df_users_reviews.drop_duplicates()

guardando el archivo como parquet

In [124]:
df_users_reviews.to_parquet(ruta_parquet_reviews)

**tratamiento y limpieza de datos**

tratamiento y limpieza archivo steam_games

In [5]:
#lectura del archivo
df_games = pd.read_parquet(ruta_parquet_games)

In [126]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,None,NaN,None
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域


Primera línea no es necesária para el dataframe

In [127]:
df_games.drop(0, inplace=True)

In [128]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,None


Normalizando la columna "price"

In [129]:
# Unifico todos los valores distintos hacia un unico valor 'Free To Play'
# Usa la función .replace() para realizar la transformación
df_games['price'] = df_games['price'].replace('Free to Play', 'Free To Play')
df_games['price'] = df_games['price'].replace('Free Demo', 'Free To Play')
df_games['price'] = df_games['price'].replace('Play for Free!', 'Free To Play')
df_games['price'] = df_games['price'].replace('Install Now', 'Free To Play')
df_games['price'] = df_games['price'].replace('Play WARMACHINE: Tactics Demo', 'Free To Play')
df_games['price'] = df_games['price'].replace('Free Mod', 'Free To Play')
df_games['price'] = df_games['price'].replace('Install Theme', 'Free To Play')
df_games['price'] = df_games['price'].replace('Third-party', 'Free To Play')
df_games['price'] = df_games['price'].replace('Play Now', 'Free To Play')
df_games['price'] = df_games['price'].replace('Free HITMAN™ Holiday Pack', 'Free To Play')
df_games['price'] = df_games['price'].replace('Play the Demo', 'Free To Play')
df_games['price'] = df_games['price'].replace('Free to Try', 'Free To Play')
df_games['price'] = df_games['price'].replace('Free to Use','Free To Play')
df_games['price'] = df_games['price'].replace('Free','Free To Play')

In [130]:
def replace_free_to_play(df):
    df["price"] = df["price"].replace("Free To Play", 0.0)
    return df


In [131]:
# Aplica la función para reemplazar 'Free To Play' por 0.0
df_games = replace_free_to_play(df_games)

In [132]:
# Limpia la columna "price" para quedarse solo con valores numéricos y decimales
df_games['price'] = df_games['price'].apply(lambda x: re.sub(r'[^\d.]', '', str(x)))
# Reemplaza cadenas vacías con NaN
df_games['price'].replace('', np.nan, inplace=True)
# Reemplazar valores nulos por 0.0 en la columna "price"
df_games["price"].fillna(0.0, inplace=True)
# Convierte la columna en tipo float
df_games['price'] = df_games['price'].astype(float)

In [133]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         32135 non-null  float64
 10  early_access  32135 non-null  object 
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


renombrando columna id para item_id

In [134]:
df_games = df_games.rename(columns = {"id":"item_id"})

In [135]:
df_games.dtypes

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price           float64
early_access     object
item_id         float64
developer        object
dtype: object

Conversión de la columna "item_id" de float a int

In [6]:
df_games['item_id'] = df_games['item_id'].astype(int)

mezclando columnas repetidas

"app_name" y "title"

In [136]:
df_games['title'].fillna(df_games['app_name'], inplace=True)

# Excluir a coluna "app_name"
df_games.drop('app_name', axis=1, inplace=True)

In [137]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   title         32133 non-null  object 
 3   url           32135 non-null  object 
 4   release_date  30068 non-null  object 
 5   tags          31972 non-null  object 
 6   reviews_url   32133 non-null  object 
 7   specs         31465 non-null  object 
 8   price         32135 non-null  float64
 9   early_access  32135 non-null  object 
 10  item_id       32133 non-null  float64
 11  developer     28836 non-null  object 
dtypes: float64(2), object(10)
memory usage: 3.2+ MB


"publisher" y "developer"

In [138]:
df_games['developer'].fillna(df_games['publisher'], inplace=True)

# Excluir a coluna "app_name"
df_games.drop('publisher', axis=1, inplace=True)

In [139]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   title         32133 non-null  object 
 2   url           32135 non-null  object 
 3   release_date  30068 non-null  object 
 4   tags          31972 non-null  object 
 5   reviews_url   32133 non-null  object 
 6   specs         31465 non-null  object 
 7   price         32135 non-null  float64
 8   early_access  32135 non-null  object 
 9   item_id       32133 non-null  float64
 10  developer     28901 non-null  object 
dtypes: float64(2), object(9)
memory usage: 2.9+ MB


Creación de la columna "year"

In [140]:
# Definir una función personalizada para extraer el año de la fecha
def extract_year(date_str):
    try:
        return int(date_str.split("-")[0])
    except (ValueError, AttributeError):
        return 2016  # Cambia 2016 al valor predeterminado que desees

# Aplicar la función personalizada a la columna "release_date"
df_games["year"] = df_games["release_date"].apply(extract_year)

# Convertir la columna "year" a tipo int
df_games["year"] = df_games["year"].astype(int)


In [141]:
df_games.head()

,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,year
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro,2018
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,643980.0,Secret Level SRL,2018
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,670290.0,Poolians.com,2017
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域,2017
88314,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,None,2016


limpiando las columnas no necesárias restantes

In [142]:
df_games = df_games.drop(columns=["url","reviews_url","early_access","specs","tags","release_date","early_access"], axis=1)

In [143]:
df_games

,genres,title,price,item_id,developer,year
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,0.00,643980.0,Secret Level SRL,2018
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,2017
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,0.99,767400.0,彼岸领域,2017
88314,None,Log Challenge,2.99,773570.0,None,2016
...,...,...,...,...,...,...
120440,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,1.99,773640.0,"Nikita ""Ghost_RUS""",2018
120441,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,4.99,733530.0,Sacada,2018
120442,"['Indie', 'Racing', 'Simulation']",Russian Roads,1.99,610660.0,Laush Dmitriy Sergeevich,2018
120443,"['Casual', 'Indie']",EXIT 2 - Directions,4.99,658870.0,"xropi,stev3ns",2017


In [144]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     28852 non-null  object 
 1   title      32133 non-null  object 
 2   price      32135 non-null  float64
 3   item_id    32133 non-null  float64
 4   developer  28901 non-null  object 
 5   year       32135 non-null  int32  
dtypes: float64(2), int32(1), object(3)
memory usage: 1.6+ MB


Eliminando valores nulos del dataframe

In [7]:
df_games = df_games.dropna(axis=0)

In [7]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28734 entries, 0 to 28733
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     28734 non-null  object 
 1   title      28734 non-null  object 
 2   price      28734 non-null  float64
 3   item_id    28734 non-null  int32  
 4   developer  28734 non-null  object 
 5   year       28734 non-null  int32  
dtypes: float64(1), int32(2), object(3)
memory usage: 1.1+ MB


In [145]:
df_games.dtypes

genres        object
title         object
price        float64
item_id      float64
developer     object
year           int32
dtype: object

Conversión del dataframe limpio en parquet

In [146]:
df_games.to_parquet(ruta_parquet_games, index=False)

tratamiento y limpieza archivo data_reviews

In [147]:
df_reviews = pd.read_parquet(ruta_parquet_reviews)

In [148]:
df_reviews.head()

,user_id,item_id,posted,recommend,review
0,76561197970982479,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,76561197970982479,43110,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,js41637,227300,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...


Normalización columna "posted"

In [149]:
def extraer_año(fecha):
    # Las fechas vienen en un formato del estilo 'Posted: July 15, 2013.'. Las fechas del año del dataset (2016) no
    # incluyen el año (por ejemplo, 'Posted: March 23.'). Usamos una expresión regular para extraer el año.
    result = re.search("\w* \d*, (\d*)", fecha)
    if result is not None:
        return int(result.group(1))
    else:
        return 2016

In [150]:
df_reviews["posted"] = df_reviews["posted"].apply(extraer_año)

In [151]:
df_reviews.head()

,user_id,item_id,posted,recommend,review
0,76561197970982479,1250,2011,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,2011,True,It's unique and worth a playthrough.
2,76561197970982479,43110,2011,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,2014,True,I know what you think when you see this title ...
4,js41637,227300,2013,True,For a simple (it's actually not all that simpl...


In [152]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58431 non-null  object
 1   item_id    58431 non-null  int64 
 2   posted     58431 non-null  int64 
 3   recommend  58431 non-null  bool  
 4   review     58401 non-null  object
dtypes: bool(1), int64(2), object(2)
memory usage: 2.3+ MB


In [153]:
df_reviews.isna().sum()

user_id       0
item_id       0
posted        0
recommend     0
review       30
dtype: int64

Conversión del dataframe limpio en parquet

In [154]:
df_reviews.to_parquet(ruta_parquet_reviews, index=False)

tratamiento y limpieza archivo data_items

In [6]:
df_items = pd.read_parquet(ruta_parquet_items)

In [156]:
df_items.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


In [157]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094092 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           int64 
 2   item_name         object
 3   playtime_forever  int64 
dtypes: int64(2), object(2)
memory usage: 194.3+ MB


In [158]:
df_items.dtypes

user_id             object
item_id              int64
item_name           object
playtime_forever     int64
dtype: object

In [159]:
df_items.isna().sum()

user_id             0
item_id             0
item_name           0
playtime_forever    0
dtype: int64

Optimización del dataframe

In [7]:
df_items = df_items[:2000000]

In [8]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           int64 
 2   item_name         object
 3   playtime_forever  int64 
dtypes: int64(2), object(2)
memory usage: 61.0+ MB


Conversión del dataframe limpio en parquet

In [160]:
df_items.to_parquet(ruta_parquet_items, index=False)

**Análisis de Sentimiento**

In [161]:
import json
import pandas as pd
import ast
import re
from textblob import TextBlob

In [162]:
df_reviews = pd.read_parquet(ruta_parquet_reviews)

In [163]:
df_reviews.head()

,user_id,item_id,posted,recommend,review
0,76561197970982479,1250,2011,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,2011,True,It's unique and worth a playthrough.
2,76561197970982479,43110,2011,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,2014,True,I know what you think when you see this title ...
4,js41637,227300,2013,True,For a simple (it's actually not all that simpl...


In [164]:
# Transformando tipo de dato float a str para la columna 'review' debido a que el análisis de sentimiento arrojaba un error por el tipo de dato
df_reviews['review'] = df_reviews['review'].astype(str)

In [165]:
# Realizamos el análisis de sentimiento para la columna 'review' de df_reviews

from textblob import TextBlob

# Función para analizar el sentimiento y asignar valores según la escala
def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    if analysis.sentiment.polarity < 0:
        return 0  # Sentimiento negativo
    elif analysis.sentiment.polarity == 0:
        return 1  # Sentimiento neutral
    else:
        return 2  # Sentimiento positivo

In [166]:
# Aplicar la función de análisis de sentimiento a la columna 'texto' y crear 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analizar_sentimiento)

In [167]:
df_reviews.head()

,user_id,item_id,posted,recommend,review,sentiment_analysis
0,76561197970982479,1250,2011,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,22200,2011,True,It's unique and worth a playthrough.,2
2,76561197970982479,43110,2011,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,251610,2014,True,I know what you think when you see this title ...,2
4,js41637,227300,2013,True,For a simple (it's actually not all that simpl...,0


In [168]:
df_reviews = df_reviews.drop("review", axis=1)

In [169]:
df_reviews.head()

,user_id,item_id,posted,recommend,sentiment_analysis
0,76561197970982479,1250,2011,True,2
1,76561197970982479,22200,2011,True,2
2,76561197970982479,43110,2011,True,2
3,js41637,251610,2014,True,2
4,js41637,227300,2013,True,0


Guardando cambios del DataFrame como parquet

In [170]:
df_reviews.to_parquet(ruta_parquet_reviews, index=False)